<a href="https://colab.research.google.com/github/vinudrago/BizCard-X/blob/main/BizCard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python
!pip install easyocr
!pip install streamlit
!pip install pyngrok
!pip install mysql.connector
!pip install streamlit_option_menu

In [1]:
%%writefile bizx1.py
import streamlit as st
from streamlit_option_menu import option_menu
import easyocr
import cv2
from PIL import Image
import numpy as np
import io
import re
import mysql.connector as sql
import pandas as pd

# Creating connection with MySQL Workbench
mydb = sql.connect(
    host="sql12.freesqldatabase.com",
    user="sql12647189",
    password="sKbwQTzLa3",
    database="sql12647189"
)
mycursor = mydb.cursor()

# TABLE CREATION
mycursor.execute("""Create table if not exists business_cards(
                        id int auto_increment primary key,
                        card_holder_name varchar(50),
                        designation varchar(50),
                        phone varchar(50),
                        mail varchar(50),
                        website varchar(50),
                        area varchar(50),
                        city varchar(50),
                        state varchar(50),
                        pincode varchar(50),
                        company_name varchar(50),
                        bin_img LONGBLOB);""")

# SETTING PAGE CONFIGURATIONS
st.set_page_config(page_title="BizCard-X", page_icon="🔍")

st.markdown(
    f"""
    <style>
    .stApp {{
        background-image: url("https://cdn.wallpapersafari.com/53/63/pnd4MG.jpg");
        background-attachment: fixed;
        background-size: cover
    }}
    </style>
    """,
    unsafe_allow_html=True
)

#Setting Title
st.title(":blue[Extracting Business Card Data] with :red[OCR: bookmark tabs:]")


with st.sidebar:
    menu = option_menu("Main Menu", ["Home","Upload","Alter","Delete"],
                      icons=["house",'cloud-upload', "list-task"],
                      menu_icon="cast",
                      default_index=1,
                      styles={"icon": {"color": "orange", "font-size": "20px"},
                              "nav-link": {"font-size": "15px", "text-align": "left", "margin": "-2px", "--hover-color": "#FFFFFF"},
                              "nav-link-selected": {"background-color": "#225154"}})

st.sidebar.title(":red[BizCard-X :card index:]")
st.sidebar.image("https://miro.medium.com/v2/format:jpg/resize:fit:1200/1*hVxkXe35kRcAht3QpJylyg.gif", width=300)

# HOME MENU
if menu == "Home":
    st.header('Welcome to Business card application')
    st.subheader(':orange[About the App:]')
    home_text = (f'''In this Streamlit web app, you can upload an image of a business
                  card and extract relevant information from it using EasyOCR. You can view,
                  modify, or delete the extracted data in this app. Additionally, the app would
                  allow users to save the extracted information into a database alongside the
                  uploaded business card image. The database would be capable of storing multiple
                  entries, each with its own business card image and the extracted information.''')

    st.markdown(f"<h4 text-align: left;'>{home_text} </h4>", unsafe_allow_html=True)
    st.subheader(":orange[Technologies Used:]")
    tech_text =('  EasyOCR, Python, MySQL, Streamlit(GUI)')
    st.markdown(f"<h4 text-align: left;'>{tech_text} </h4>", unsafe_allow_html=True)

# UPLOAD MENU
elif menu == "Upload":
    st.subheader(":blue[Upload a Business Card]")
    image = st.file_uploader('Upload an image', type=['jpg', 'jpeg', 'png'])
    if image is not None:
        col1, col2 = st.columns([1, 1])
        with col1:
            st.success("Image Uploaded Successfully")
            st.image(image)
        with col2:
            with st.spinner("Please wait processing image..."):
                image_bytes = image.read()
                reader = easyocr.Reader(['en'], gpu=False)
                result = reader.readtext(image_bytes)

                pil_image = Image.open(io.BytesIO(image_bytes))
                img = np.array(pil_image)
                for i in result:
                    top_left = tuple(map(int,i[0][0]))
                    bottom_right = tuple(map(int,i[0][2]))
                    text = i[1]
                    font = cv2.FONT_HERSHEY_SIMPLEX

                    img = cv2.rectangle(img,top_left,bottom_right,(0,0,255),5)
                    img = cv2.putText(img,text,top_left,font,1,(0,255,0),2,cv2.LINE_AA)
                st.success('Got the details')
                st.image(img, channels="BGR")
        def img_to_binary(file):
            pil_image = Image.open(image)
            with io.BytesIO() as buffer:
                pil_image.save(buffer, format="PNG")
                binary_image = buffer.getvalue()
            return binary_image
        data = {'card_holder_name':[],'designation':[],'phone': [],'mail':[],'website': [],'area': [],'city': [],'state':[],
                'pincode':[],'company_name':[],'binary_image':img_to_binary(image)}

        def get_data(result):
            for ind,i in enumerate(result):
                # card_holder
                if ind == 0:
                    data['card_holder_name'] = i[1]
                #designation
                elif ind == 1:
                    data['designation'] = i[1]
                # email
                elif '@' in i[1]:
                    data['mail'].append(i[1])
                #phone
                elif '-' in i[1]:
                    data['phone'].append(i[1])
                    if len(data['phone']) >1:
                        data['phone'] = ' & '.join(data['phone'])
                # website
                elif 'www ' in i[1].lower() or 'www.' in i[1].lower():
                    data['website'] = i[1]
                elif 'WWW' in i[1]:
                    data['website'] =result[4][1]+'.'+result[5][1]
                # Area
                elif re.fullmatch('^[0-9]+ [a-zA-Z]+$',i[1]): #2
                    data['area'] = i[1]
                    data['city'] = result[7][1]
                elif re.findall('^[0-9]+ [a-zA-Z ]+,, [a-zA-Z ,;]+',i[1]): #4
                    r = ' '.join(re.findall('^[0-9]+ [a-zA-Z ]+,, [a-zA-Z ,;]+',i[1]))
                    data['area'] = ' '.join(r.split(' ')[0:3])
                    data['city'] = ''.join(r.split(' ')[-2])
                    data['state'] = ''.join(r.split(' ')[-1])
                elif re.findall('^[0-9]+ [a-zA-Z ,]+; [a-zA-Z]+',i[1]): #5
                    data['area'] = ' '.join(i[1].split(' ')[0:3])
                    data['city'] = i[1].split(' ')[-2]
                    data['state'] = i[1].split(' ')[-1]
                elif re.findall('[0-9]+ [a-zA-Z]+ [a-zA-Z]+ , [a-zA-Z]+',i[1]): #1
                    data['area'] = ' '.join(i[1].split(' ')[0:3])
                    data['city'] = i[1].split(' ')[-1]
                # state
                elif re.findall('^[a-zA-Z]+ [0-9]+', i[1]):
                    data['state'] = i[1].split()[0]
                    data['pincode'] = i[1].split()[-1]
                elif re.findall('[0-9]+', i[1]):
                    data['pincode'] = i[1]
                #company
                elif ind == len(result)-1 and i[1].isupper():
                    data['company_name'] = result[-2][1]+' '+result[-1][1]
                elif ind == len(result)-1 and len(i[1])>10:
                    data['company_name'] = result[-1][1]
                elif ind == len(result)-1 and len(i[1])>5:
                    data['company_name'] = result[-3][1]+' '+result[-1][1]
                elif ind == len(result)-1:
                    data['company_name'] = result[-4][1]+' '+result[-2][1]
        get_data(result)
        st.markdown('### Extracted data')
        df = pd.DataFrame(data)
        st.dataframe(df)
        if st.button("Upload to DataBase"):
            for i, row in df.iterrows():
                query = "INSERT INTO business_cards(card_holder_name,designation,phone,mail,website,area,city,state,pincode,company_name,bin_img) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                mycursor.execute(query,tuple(row))
                mydb.commit()
            st.success("Successfully Uploaded")

        if st.button(":blue[View updated data]"):
            mycursor.execute("select card_holder_name,designation,phone,mail,website,area,city,state,pincode,company_name from business_cards")
            updated_df = pd.DataFrame(mycursor.fetchall(),
                                          columns=["card_holder_name", "designation", "phone", "mail",
                                                   "website", "area", "city", "state", "pincode", "company_name"])

# ALTER MENU
elif menu == "Alter":
    st.markdown(":blue[Alter the data here]")
    mycursor.execute('Select card_holder_name from business_cards')
    names = ["None"]  # Add "None" option to the list of names
    for i in mycursor.fetchall():
        names.append(i[0])

    # Display the selectbox with the "None" option
    selected_card = st.selectbox('Select One', options=names)

    if selected_card == "None":
        st.write("No card selected.")
    else:
        st.markdown(f"#### Update or modify data for: {selected_card}")

    mycursor.execute(f"Select card_holder_name,designation,phone,mail,website,area,city,state,pincode,company_name from business_cards where card_holder_name = '{selected}'")
    result = mycursor.fetchone()

    card_holder_name = st.text_input("card_holder_name",result[0])
    designation = st.text_input("designation", result[1])
    phone = st.text_input("phone", result[2])
    mail = st.text_input("mail", result[3])
    website = st.text_input("website", result[4])
    area = st.text_input("area", result[5])
    city = st.text_input("city", result[6])
    state = st.text_input("state", result[7])
    pincode = st.text_input("pincode", result[8])
    company_name = st.text_input("company_name", result[9])

    if st.button("Commit changes to DB"):
        mycursor.execute("update business_cards set card_holder_name=%s,designation=%s,phone=%s,mail=%s,website=%s,"
                          "area=%s,city=%s,state=%s,pincode=%s,company_name=%s where card_holder_name = %s",
                          (card_holder_name,designation,phone,mail,website,area,city,state,pincode,company_name,selected))
        mydb.commit()
        st.success('Updated successfully')


# DELETE MENU
elif menu == 'Delete':
    mycursor.execute('Select card_holder_name from business_cards')
    names = ["None"]  # Add "None" option to the list of names
    for i in mycursor.fetchall():
        names.append(i[0])

    # Display the selectbox with the "None" option
    selected_card = st.selectbox('Select One', options=names)

    if selected_card == "None":
        st.write("No card selected.")
    else:
        st.markdown(f"#### Update or modify data for: {selected_card}")
    st.markdown(f"You selected {name}")
    st.markdown("Do you want to proceed?")
    if st.button("Yes"):
        mycursor.execute(f"Delete From business_cards where card_holder_name = '{name}'")
        mydb.commit()
        st.success("successfully deleted")


Writing bizx1.py


In [ ]:
! pip install streamlit -q

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
! streamlit run bizx1.py & npx localtunnel --port 8501